# Knowledge Graph

## Einführung
Im 4. Assignment wird ein Knowledge Graph implementiert, und die bisherigen RDF-Daten mit neuen Informationen angereichert, die über externe Links und Quellen abgefragt werden. Außerdem wird die von uns erstellte lokale Suchmaschine, die wir in Assignment 2 entwickelt haben, erweitert - hierzu wird das von uns entwickelte Ontologie-Modell verwendet, welches in Assignment 3 implementiert wurde.
Zum Schluss werden mithilfe von SPARQL semantische Abfragen erstellt, durch die wir Informationen über den Damen-Volleyballkader erhalten können.

## Arbeitsaufteilung
* Cesar Laura:
* Dilly Julian:
* Ecker Annina: 1. Integration Ontology with Search Engine


### Installation und Importieren der benötigten Libraries

In [ ]:
!pip install -r requirements.txt

In [ ]:
import pandas as pd
import text_processing as tp
import tfidf_search as ts
import ontology_graph as og

### 1. Integration der Ontologie in die lokale Suchmaschine

In [ ]:
spielerinnen_csv = pd.read_csv("./Spielerinnendaten.csv")

# text_processing + Ontologie einfügen
inverted_index = tp.build_inverted_index(spielerinnen_csv.values)

#### 1.1 Änderung 
Bei der Implementierung des bisherigen TFIDF-Scores hat sich gezeigt, dass `cm` als eigener Token interpretiert wurde, und nicht als Teil der Information zur Körpergröße:
<details>
  <summary>Screenshot: Available Tokens, Erstversion</summary>
  
  ![title](available_tokens.png)
  
</details>

Daher wurde die Logik für die Funktion `clean_text`angepasst, wo mithilfe von RegEx Einheit und Zahl durch einen `_` kombiniert werden.

<details>
    <summary>Code: Before & After</summary>
    
![title](height_token_solution.png)
  
</details>

**Diese Änderung wurde jedoch wieder rückgängig gemacht, da im weiteren Verlauf der Implementierung bessere Suchanfragen mittels SPARQL erstellt werden sollen.**

<details>
    <summary>Suchanfrage mit geänderter RegEx-Abfrage (wurde wieder rückgängig gemacht!):</summary>
    
![title](suche_w_regex.png)
  
</details>

---

In [ ]:
tfidf_df = ts.calculate_tfidf(spielerinnen_csv.values, inverted_index)

# Check aller Tokens:
# Zeige die existierenden Tokens in der TF-IDF Tabelle
print(f"Available tokens in the tfidf_df Dataframe:{tfidf_df.columns}")

---

In [ ]:
# Suchabfrage: Test 1
# Check der AND-Bedingung
ts.search(tfidf_df, "Diagonal and 185 and cm", spielerinnen_csv)

volleyball_graph = og.create_volleyball_ontology()

---

Für die OR-Bedingung wurde ein `outer`-Merge verwendet und implementiert, für AND-Bedingungen ein `inner`-Merge:


In [ ]:
# Suchabfrage: Test 2
# Check der OR-Bedingung
ts.search(tfidf_df, "Außenangriff or Nina", spielerinnen_csv)

volleyball_graph = og.create_volleyball_ontology()

---

In [ ]:
# Suchabfrage: Test 3
# Anfrage Spielerin mit zugehöriger Position, kein OR/AND

ts.search(tfidf_df, "Nicole Aufspiel", spielerinnen_csv)

volleyball_graph = og.create_volleyball_ontology()

---

In [ ]:
# Suchabfrage: Test 4
# Anfrage Spielerin ohne zugehörige Position, kein OR/AND

ts.search(tfidf_df, "Nicole Libero", spielerinnen_csv)

volleyball_graph = og.create_volleyball_ontology()

---


Der bisherige Code aus `text_processing.py`, `tfidf_search.py` und `ontology_graph.py` wurde abgewandelt und die Onthologie eingefügt. <br>Zusätzlich wurde die Ausgabe so formatiert, dass man den Score direkt mit der jeweiligen Spielerin gemeinsam ausgegeben erhält. Die Onthologie ist zwar ab diesem Punkt bereits vorhanden, allerdings wird nach wie vor mittels Text-Token gesucht.<br>
Es ist noch nicht möglich, z.B. den Namen einer Spielerin und eine beliebige Position abzufragen (siehe **Suchabfrage Test 4**). <br>Um die Suchfunktion zu erweitern, ist **SPARQL** notwendig, da so die Onthologie-Daten tatsächlich verarbeitet werden können. Zunächst werden allerdings noch die RDF-Daten über externe Links und Quellen angereichert.

---

### 2. WIP

In [ ]:
display(spielerinnen_csv)

### 3. WIP

# Ressourcen & Source-Docs

## Inverted Index, Onthology and Search
1. [NLP Stanford - A first Take at Building an Inverted Index](https://nlp.stanford.edu/IR-book/html/htmledition/a-first-take-at-building-an-inverted-index-1.html)
2. Quasi alle vorherigen Quellen, die angegeben worden sind

## WIP
1. [Link](https://www.google.com/)
2. [Link](https://www.google.com/)
3. [Link](https://www.google.com/)
4. [Link](https://www.google.com/)
5. [Link](https://www.google.com/)

## WIP
1. [Link](https://www.google.com/)
2. [Link](https://www.google.com/)
3. [Link](https://www.google.com/)
4. [Link](https://www.google.com/)
5. [Link](https://www.google.com/)

# Challenges bei der Implementierung


#### Cesar Laura,

#### Dilly Julian,


#### Ecker Annina, Integrierung der Onthologie in die Suche:
Grundsätzlich war unsere bestehende Logik schon sehr gut. Ich war mir nur sehr unsicher, inwiefern bereits der erste Teil "more sophisticaed" sein muss, weswegen ich mich nochmal genauer in Inverted Indices usw. eingelesen habe, und für die Suche die Abfragen zusätzlich verfeinert habe. Alles aber für Text-Token, da SPARQL semantisch deutlich bessere Abfragen machen kann, was ich gelesen habe.
